In [1]:
using DataFrames, CSV, Dates

In [2]:
df = CSV.read("POZOS_COMPILADO_CORR.csv", DataFrame) ;

In [3]:
# Renaming the columns
old_names = ["Identificador", "Nombre_del_pozo", "Asignaci\xf3n_o_Contrato", "Hidrocarburos_Liquidos_(Mbd)", 
            "Petr\xf3leo_(Mbd)", "Condensado_(Mbd)", "Gas_asociado_(MMpcd)", "Gas_no_asociado_(MMpcd)", "Agua_(Mbd)"]
new_names = ["id", "Pozo", "Contrato", "Ql_(Mbd)", "Qo_(Mbd)", "Cond_(Mbd)", "Qga_(MMpcd)", "Qgna_(MMpcd)", 
            "Qw_(Mbd)"]
#rename!(df, old_names .=> new_names)

# Changing the dates
#df.Fecha = Date.(df.Fecha, "dd-mm-yyyy")
show(first(df, 5), allcols=true)

5×11 DataFrame
 Row │ Fecha       id          Pozo         Cuenca            Contrato  Ql_(Mbd)  Qo_(Mbd)  Cond_(Mbd)  Qga_(MMpcd)  Qgna_(MMpcd)  Qw_(Mbd) 
     │ Date        Int64       String       String31          String?   Float64   Float64   Float64     Float64      Float64       Float64  
─────┼──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ 1930-06-01  1000023609  Poza Rica-2  TAMPICO-MISANTLA  missing   0.009015  0.009015         0.0     0.00418            0.0       0.0
   2 │ 1930-07-01  1000023609  Poza Rica-2  TAMPICO-MISANTLA  missing   0.020695  0.020695         0.0     0.009594           0.0       0.0
   3 │ 1930-08-01  1000023609  Poza Rica-2  TAMPICO-MISANTLA  missing   0.020695  0.020695         0.0     0.009594           0.0       0.0
   4 │ 1930-09-01  1000023609  Poza Rica-2  TAMPICO-MISANTLA  missing   0.020756  0.020756         0.0     0.009622           0.0       0.0
  

In [7]:
size(df)

(3254852, 11)

In [8]:
# Fucntion to forward fill missing values
ffill(v) = v[accumulate(max, [i*!ismissing(v[i]) for i in 1:length(v)], init=1)]

tablero = DataFrame()
pozos = [pozo for pozo in unique(df.:Pozo)]
cont = 0
tot = length(pozos)
fecha_reporte = Date(2024, 03, 01)

for pozo in pozos
    # Delelete duplicates
    df_pozo = df[df.Pozo .== pozo, :]
    # unique!(df_pozo, :Fecha)

    # Merge dataframes to fill the date gaps
    if year(fecha_reporte) - year(df_pozo.Fecha[end]) > 10
        d = collect(Date(df_pozo.Fecha[1]):Month(1):df_pozo.Fecha[end] + Year(1))

    else
        d = collect(Date(df_pozo.Fecha[1]):Month(1):fecha_reporte)
    end
    d_index = DataFrame(Fecha = d)
    df_pfinal = leftjoin(d_index, df_pozo; on=:Fecha)
    sort!(df_pfinal, :Fecha)
    df_pfinal[!, 1:5] = DataFrame([ffill(df_pfinal[!, c]) for c in names(df_pfinal[!, 1:5])], names(df_pfinal[!, 1:5]))
    df_pfinal[!, 6:end] = coalesce.(df_pfinal[!, 6:end], 0)

    # Create the Cumulative columns
    cols = ["Qo_(Mbd)", "Cond_(Mbd)", "Qga_(MMpcd)", "Qgna_(MMpcd)", "Qw_(Mbd)"]
    new_cols = ["Qo_ac(Mbd)", "Cond_ac(Mbd)", "Qga_ac(MMpcd)", "Qgna_ac(MMpcd)", "Qw_ac(Mbd)"]

    for (new_col, col) in zip(new_cols, cols)
        df_pfinal[!, new_col] = cumsum(df_pfinal[!, col])
    end

    # tablero = vcat([tablero, df_pfinal])
    append!(tablero, df_pfinal)
    cont += 1
    perc = round((cont/tot)*100, digits=3)
    println("Pozo $cont de $tot : $perc %")
end

Pozo 1 de 33895 : 0.003 %
Pozo 2 de 33895 : 0.006 %
Pozo 3 de 33895 : 0.009 %
Pozo 4 de 33895 : 0.012 %
Pozo 5 de 33895 : 0.015 %
Pozo 6 de 33895 : 0.018 %
Pozo 7 de 33895 : 0.021 %
Pozo 8 de 33895 : 0.024 %
Pozo 9 de 33895 : 0.027 %
Pozo 10 de 33895 : 0.03 %
Pozo 11 de 33895 : 0.032 %
Pozo 12 de 33895 : 0.035 %
Pozo 13 de 33895 : 0.038 %
Pozo 14 de 33895 : 0.041 %
Pozo 15 de 33895 : 0.044 %
Pozo 16 de 33895 : 0.047 %
Pozo 17 de 33895 : 0.05 %
Pozo 18 de 33895 : 0.053 %
Pozo 19 de 33895 : 0.056 %
Pozo 20 de 33895 : 0.059 %
Pozo 21 de 33895 : 0.062 %
Pozo 22 de 33895 : 0.065 %
Pozo 23 de 33895 : 0.068 %
Pozo 24 de 33895 : 0.071 %
Pozo 25 de 33895 : 0.074 %
Pozo 26 de 33895 : 0.077 %
Pozo 27 de 33895 : 0.08 %
Pozo 28 de 33895 : 0.083 %
Pozo 29 de 33895 : 0.086 %
Pozo 30 de 33895 : 0.089 %
Pozo 31 de 33895 : 0.091 %
Pozo 32 de 33895 : 0.094 %
Pozo 33 de 33895 : 0.097 %
Pozo 34 de 33895 : 0.1 %
Pozo 35 de 33895 : 0.103 %
Pozo 36 de 33895 : 0.106 %
Pozo 37 de 33895 : 0.109 %
Pozo 38 de 3389

Excessive output truncated after 524305 bytes.


Pozo 17387 de 33895 : 51.297 %

In [7]:
# show(tablero, allrows=true, allcols=true)

In [8]:
# using Plots

# Plots.scatter(tablero[!, "Fecha"], tablero[!, "Qo_(Mbd)"], size=(1000, 400))
# plot!(tablero[!, "Fecha"], tablero[!, "Qo_(Mbd)"])


In [9]:
tablero.id = Int64.(tablero[!, :id])
tablero.Pozo = String63.(tablero[!, :Pozo])
tablero.Cuenca = String63.(tablero[!, :Cuenca])
tablero.Contrato = [ismissing(x) ? "SIN CONTRATO" : String63(x) for x in tablero.:Contrato]
# last(tablero, 3)

5884901-element Vector{AbstractString}:
 "SIN CONTRATO"
 "SIN CONTRATO"
 "SIN CONTRATO"
 "SIN CONTRATO"
 "SIN CONTRATO"
 "SIN CONTRATO"
 "SIN CONTRATO"
 "SIN CONTRATO"
 "SIN CONTRATO"
 "SIN CONTRATO"
 ⋮
 "AE-0148-2M - UCHUKIL"
 "AE-0009-5M - TUCOO-XAXAMANI - 01"
 "A-0004-4M - CAMPO AGUA FRIA"
 "AE-0141-3M - COMALCALCO"
 "AR-0449-2M - CAMPO PRESIDENTE ALEMAN"
 "AE-0045-7M - AGUA DULCE - 04"
 "A-0032-2M - CAMPO AYATSIL"
 "AE-0009-5M - TUCOO-XAXAMANI - 01"
 "CNH-A4.OGARRIO/2018"

In [10]:
size(tablero)

(5884901, 16)

In [19]:
names(tablero)[6:end]

11-element Vector{String}:
 "Ql_(Mbd)"
 "Qo_(Mbd)"
 "Cond_(Mbd)"
 "Qga_(MMpcd)"
 "Qgna_(MMpcd)"
 "Qw_(Mbd)"
 "Qo_ac(Mbd)"
 "Cond_ac(Mbd)"
 "Qga_ac(MMpcd)"
 "Qgna_ac(MMpcd)"
 "Qw_ac(Mbd)"

In [23]:
using Printf

transform(col, val) = val
transform(col, val::Float64) = @sprintf("%.3f", val)
CSV.write("tablero_julia.csv", tablero, encoding=:latin1, transform=transform)

"tablero_julia.csv"

In [19]:
using CSV, DataFrames

tab = CSV.read("tablero_julia.csv", DataFrame) ;

In [32]:
length(unique(tab.Pozo))

1